In [1]:
import paho.mqtt.client as mqtt
import threading
from datetime import datetime
import json
import time
from postgres_db import PostgresDatabase

class MQTTSubscriber(threading.Thread):
    def __init__(self, broker, port, user, password, topics, db_handler):
        super().__init__()
        self.broker = broker
        self.port = port
        self.user = user
        self.password = password
        self.topics = topics
        self.db_handler = db_handler
        self.client = mqtt.Client()
        self.client.username_pw_set(self.user, self.password)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.running = True

    def on_connect(self, client, userdata, flags, rc):
        print("Conectado com sucesso. Código de resultado:", rc)
        for topic in self.topics:
            client.subscribe(topic)
            print(f"Inscrito no tópico: {topic}")

    def on_message(self, client, userdata, msg):
        if self.running:
            timestamp = datetime.now()
            payload = msg.payload.decode()
            print(f"Recebido {payload} do tópico {msg.topic}")

            if msg.topic.endswith("cpu_frequency"):
                self.db_handler.insert_cpu_frequency(timestamp, payload)
            elif msg.topic.endswith("cpu_usage"):
                self.db_handler.insert_cpu_usage(timestamp, payload)
            elif msg.topic.endswith("memory_usage"):
                self.db_handler.insert_memory_usage(timestamp, payload)
            elif msg.topic.endswith("network_io"):
                data = json.loads(payload)
                self.db_handler.insert_network_io(timestamp, data["n_input"], data["n_output"])

    def run(self):
        self.client.connect(self.broker, self.port, 60)
        self.client.loop_forever()

    def stop(self):
        self.running = False
        self.client.disconnect()





if __name__ == "__main__":
    # Configurações do MQTT
    MQTT_BROKER = 'www.maqiatto.com'
    MQTT_PORT = 1883
    MQTT_USER = 'warleyxavier.fernandes@gmail.com'
    MQTT_PASSWORD = 'wZn&y6Pt'

    # Configuração do Banco de Dados PostgreSQL
    db_handler = PostgresDatabase("init.sql")

    # Tópicos para assinar
    TOPICS = [
        "warleyxavier.fernandes@gmail.com/device_1/cpu_frequency",
        "warleyxavier.fernandes@gmail.com/device_1/cpu_usage",
        "warleyxavier.fernandes@gmail.com/device_1/memory_usage",
        "warleyxavier.fernandes@gmail.com/device_1/network_io"
    ]

    # Criar e iniciar o subscriber
    subscriber = MQTTSubscriber(MQTT_BROKER, MQTT_PORT, MQTT_USER, MQTT_PASSWORD, TOPICS, db_handler)
    subscriber.start()

    try:
        # Manter o programa em execução
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Interrompido pelo usuário")
        subscriber.stop()

    finally:
        # Fechar a conexão com o banco de dados
        db_handler.close()

C:\Users\Warle\AppData\Local\Temp\ipykernel_26112\665720026.py:17: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  self.client = mqtt.Client()


Conectado com sucesso. Código de resultado: 0
Inscrito no tópico: warleyxavier.fernandes@gmail.com/device_1/cpu_frequency
Inscrito no tópico: warleyxavier.fernandes@gmail.com/device_1/cpu_usage
Inscrito no tópico: warleyxavier.fernandes@gmail.com/device_1/memory_usage
Inscrito no tópico: warleyxavier.fernandes@gmail.com/device_1/network_io
Recebido 2688.0 do tópico warleyxavier.fernandes@gmail.com/device_1/cpu_frequency
Recebido 69.8 do tópico warleyxavier.fernandes@gmail.com/device_1/memory_usage
Recebido {"n_input": 1024, "n_output": 2048} do tópico warleyxavier.fernandes@gmail.com/device_1/network_io
Recebido 7.8 do tópico warleyxavier.fernandes@gmail.com/device_1/cpu_usage
Recebido 2688.0 do tópico warleyxavier.fernandes@gmail.com/device_1/cpu_frequency
Recebido 70.2 do tópico warleyxavier.fernandes@gmail.com/device_1/memory_usage
Recebido {"n_input": 1024, "n_output": 2048} do tópico warleyxavier.fernandes@gmail.com/device_1/network_io
Recebido 2.1 do tópico warleyxavier.fernandes

AttributeError: 'PostgresDatabase' object has no attribute 'close'